In [1]:
!pip install transformers[torch] accelerate
!pip install datasets
!pip install sentence_transformers
!pip install numba



In [2]:

import pandas as pd
import numpy as np
from numba import cuda
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm import tqdm
import torch

df1=pd.read_csv('preprocessedexp_dataset.csv')
df1.head()
df1.drop(columns=['File Name','Transcript','tokenized_Transcript'],inplace=True)


In [3]:
import spacy
from collections import Counter
!python -m spacy download en_core_web_lg
# Load Spacy's English language model
nlp = spacy.load("en_core_web_lg")
# Define a function to tokenize text
def tokenize(txt):
    # Remove punctuation and double space, and convert to lowercase:
    tokens = [token.text.lower() for token in txt if not (token.is_punct or token.is_space)]
    # Join tokens back into a string
    tokens = " ".join(tokens)
    return tokens



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:

# Apply the preprocessing pipeline using nlp.pipe
tokenized_txt = []

for txt in nlp.pipe(df1['clean_text']):
    tokenized_txt.append(tokenize(txt))


In [5]:
# Add the preprocessed text as a new column in the dataframe
df1['tokenized_Transcript'] = tokenized_txt

df1.drop(columns=['clean_text','Unnamed: 0'],inplace=True)

df1['Label'] = df1['Label'].map({'truthful': 1, 'decpetive': 0})
df1['Label'].fillna(0, inplace=True)
df1['Label'] = df1['Label'].astype(int)
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Preprocess function to tokenize input text and format labels
def preprocess_function(examples):
    inputs = examples["tokenized_Transcript"]
    targets = examples["Label"]

    # Convert labels to lists
    targets = [[label] for label in targets]

    # Ensure that inputs are lists of strings
    if isinstance(inputs, str):
        inputs = [inputs]

    # Tokenize inputs
    inputs_tokenized = tokenizer(inputs, padding=True, truncation=True)['input_ids']

    return {'input_ids': inputs_tokenized, 'labels': targets}


In [6]:
def create_dataset(df1, seed=42, cv=10):
    train = {f'split_{i+1}': [] for i in range(cv)}
    test = {f'split_{i+1}': [] for i in range(cv)}

    np.random.seed(seed)

    for k in train.keys():
        train[k] = df1.sample(frac=0.8, random_state=seed)
        test[k] = df1.drop(train[k].index)

    return train, test
seed = 42
numcv = 10
num_epochs = 3
model_size = 'small'


In [7]:
#!pip install --upgrade transformers accelerate
import accelerate
from sklearn.metrics import accuracy_score
train, test = create_dataset(df1, seed=seed, cv=numcv)
results = {}
collect_result = {}
ground_truth = {}
# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"


In [11]:
from transformers import AutoModelForSeq2SeqLM
for sp in tqdm(train.keys()):
    data_train = Dataset.from_pandas(train[sp])
    data_test = Dataset.from_pandas(test[sp])
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
    # Tokenize and preprocess data
    data_train = data_train.map(preprocess_function, batched=True)
    data_test = data_test.map(preprocess_function, batched=True)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",  # The output directory
        logging_dir="./logs",
        evaluation_strategy="epoch",  # Evaluation is done at the end of each epoch
        learning_rate=5e-4,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
    )
    # Define trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset=data_test,
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()
    # Get predictions
    predictions = trainer.predict(data_test)
    pred_labels = np.argmax(predictions.predictions, axis=1)

    # Store predictions in the original DataFrame
    df1.loc[test[sp].index, 'Prediction'] = pred_labels

    # Clean up
    del model
    torch.cuda.empty_cache()

# Display final DataFrame with predictions
print("Final DataFrame with Predictions:")
print(df1)


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.943781
2,3.324400,0.769494
3,1.436700,0.706287


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


100%|██████████| 10/10 [19:31<00:00, 117.12s/it]

Final DataFrame with Predictions:
      Label                               tokenized_Transcript  Prediction
0         1  recently visit paris moment feel like scene ro...         NaN
1         1  step fruit u s o time accelerate travel san fr...         1.0
2         1  trip u s incredible landing new york city feel...         NaN
3         1  visit usa time like enter new world mention me...         NaN
4         1  step plane usa time fill anticipation visit wa...         1.0
...     ...                                                ...         ...
1021      0  travel experience visit dark hedge northern ir...         NaN
1022      0  travel experience visit muchu picchu peru actu...         NaN
1023      0  travel experience visit hallstatt austria actu...         NaN
1024      0  trip visit jiuzhaigou valley china actually na...         NaN
1025      0  trip visit bagan myanmar actually old city bud...         NaN

[1026 rows x 3 columns]


In [13]:

# Assuming ground_truth is defined outside the loop
# Calculate accuracy
ground_truth = df1.loc[test[sp].index, 'Label']
test_accuracy = accuracy_score(ground_truth, df1.loc[test[sp].index, 'Prediction'])
print("Test Accuracy:",test_accuracy)

Test Accuracy: 0.6487804878048781
